In [1]:
import brightway2 as bw
import numpy as np
import json, os

#Local files
from import_databases import *

Using environment variable BRIGHTWAY2_DIR for data directory:
/data/user/kim_a/Brightway3


In [2]:
with open('global_settings.json', 'rb') as f:
    settings = json.load(f)
which_pc = settings['which_pc']
which_pc

'merlin'

### TODO -> give paths to databases files

In [3]:
if which_pc == 'local':
    # Older databases
    ei33_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.3 cutoff/datasets'  
    ex22_path = '/Users/akim/Documents/LCA_files/exiobase 2.2/mrIOT_IxI_fpa_coefficient_version2.2.2'
    ag12_path = '/Users/akim/Documents/LCA_files/agribalyse 1.2/AGRIBALYSEv1.2_Complete.xml'
    # Newer databases
    ei36_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.6 cutoff/datasets'  
    # ex34_path = '/Users/akim/Documents/LCA_files/exiobase 3.4/IOT_2011_pxp/' #TODO
    ag13_path = '/Users/akim/Documents/LCA_files/agribalyse 1.3/Agribalyse CSV FINAL_no links_Nov2016v3.CSV'
    habe_path = '/Users/akim/Documents/LCA_files/HABE_2017/'
    co_path = 'data/es8b01452_si_002.xlsx'
elif which_pc == 'merlin':
    ex22_path = '/data/user/kim_a/LCA_files/exiobase_22/mrIOT_IxI_fpa_coefficient_version2.2.2'
    ei33_path = '/data/user/kim_a/LCA_files/ecoinvent_33_cutoff/datasets'  
    ei36_path = '/data/user/kim_a/LCA_files/ecoinvent_36_cutoff/datasets'  
    ag13_path = '/data/user/kim_a/LCA_files/agribalyse_13/Agribalyse CSV FINAL_no links_Nov2016v3.CSV'
    habe_path = '/data/user/kim_a/LCA_files/HABE_2017'
    co_path = 'data/es8b01452_si_002.xlsx'

### TODO -> choose project

In [4]:
project = 'GSA for oases'
bw.projects.set_current(project)
# bw.projects.delete_project(project, delete_dir=True)

In [5]:
bw.databases

Databases dictionary with 0 objects

In [ ]:
del bw.databases["Agribalyse 1.3 - ecoinvent 3.6 cutoff - new biosphere"]
del bw.databases["Agribalyse 1.3 - ecoinvent 3.6 cutoff"]

### Import necessary databases

In [6]:
%%time 
ei36_name = 'ecoinvent 3.6 cutoff'
co_name = CONSUMPTION_DB_NAME

if project == 'GSA for ecoinvent':
    bw.bw2setup()
    import_exiobase_22(ex22_path)
    import_ecoinvent(ei36_path, ei36_name)
    import_agribalyse_13(ag13_path, ei36_name)
    # Consumption DB
    create_ecoinvent_33_project(ei33_path)
    import_consumption_db(co_path, habe_path, exclude_dbs=['heia'])
    add_consumption_activities(co_name, habe_path)

elif project == 'GSA for paper':
    bw.bw2setup()
    import_ecoinvent(ei36_path, ei36_name)
    # Consumption DB
    create_ecoinvent_33_project(ei33_path)
    exclude_dbs = [
        'heia', 
        'EXIOBASE 2.2',
        'Agribalyse 1.2',
        'Agribalyse 1.3 - ecoinvent 3.6 cutoff',
    ]
    import_consumption_db(co_path, habe_path, exclude_dbs=exclude_dbs)
    add_consumption_activities(co_name, habe_path)
    add_consumption_categories(co_name, co_path)
    add_consumption_sectors(co_name)
    
elif project == 'GSA no exiobase':
    bw.bw2setup()
    import_ecoinvent(ei36_path, ei36_name)
    import_agribalyse_13(ag13_path, ei36_name)
    # Consumption DB
    create_ecoinvent_33_project(ei33_path)
    exclude_dbs = [
        'heia', 
        'EXIOBASE 2.2',
    ]
    import_consumption_db(co_path, habe_path, exclude_dbs=exclude_dbs)
    add_consumption_activities(co_name, habe_path)
    
elif project == 'GSA for setac':
    bw.bw2setup()
    import_ecoinvent(ei36_path, ei36_name)
    # Consumption DB
    create_ecoinvent_33_project(ei33_path)
    exclude_dbs = [
        'heia', 
        'EXIOBASE 2.2',
        'Agribalyse 1.2',
        'Agribalyse 1.3 - ecoinvent 3.6 cutoff',
    ]
    import_consumption_db(co_path, habe_path, exclude_dbs=exclude_dbs)
    add_consumption_activities(co_name, habe_path)
    add_consumption_categories(co_name, co_path)
    add_consumption_sectors(co_name)
    
elif project == 'GSA for oases':
    bw.bw2setup()
    import_ecoinvent(ei36_path, ei36_name)
    # Consumption DB
    create_ecoinvent_33_project(ei33_path)
    import_agribalyse_13(ag13_path, ei36_name)
    exclude_dbs = [
        'heia', 
        'EXIOBASE 2.2',
    ]
#     import_consumption_db(co_path, habe_path, exclude_dbs=exclude_dbs)
#     add_consumption_activities(co_name, habe_path)
#     add_consumption_categories(co_name, co_path)
#     add_consumption_sectors(co_name)

Creating default biosphere



Writing activities to SQLite3 database:


Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applied 2 strategies in 0.00 seconds


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 12/02/2020 12:30:48
  Finished: 12/02/2020 12:30:48
  Total time elapsed: 00:00:00
  CPU %: 58.70
  Memory %: 0.04
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 4 strategies in 0.78 seconds
Wrote 850 LCIA methods with 219059 characterization factors
Creating core data migrations

Extracting XML data from 18121 datasets
Extracted 18121 datasets in 38.48 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create

...

Title: Writing activities to SQLite3 database:
  Started: 12/02/2020 12:32:32
  Finished: 12/02/2020 12:33:32
  Total time elapsed: 00:00:59
  CPU %: 51.30
  Memory %: 0.40
Created database: ecoinvent 3.6 cutoff
Biosphere database already present!!! No setup is needed
ecoinvent 3.3 cutoff database already present!!! No import is needed
Extracted 1189 unallocated datasets in 2.65 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: assign_only_product_as_production
Applying strategy: drop_unspecified_subcategories
Applying strategy: sp_allocate_products
Applying strategy: fix_zero_allocation_products
Applying strategy: split_simapro_name_geo
Applying strategy: strip_biosphere_exc_locations
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: set_code_by_activity_hash
Applying strategy: link_technosphere_based_on_name_unit_location
Applying strategy: change_electricity_unit_mj_to_kwh
Applying st

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 12/02/2020 12:47:55
  Finished: 12/02/2020 12:47:55
  Total time elapsed: 00:00:00
  CPU %: 91.90
  Memory %: 0.24
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: link_iterable_by_fields
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
{'uncertainty type': 0, 'scale': nan, 'loc': nan, 'negative': False, 'amount': 0.0234, 'categories': ('Materials/fuels',), 'name': 'market for diesel', 'unit': 'kilogram', 'comment': '', 'type': 'technosphere', 'location': 'CH', 'reference product': 'diesel', 'system model': 'Allocation, cut-off by classification', 'simapro name': 'Diesel {CH}| market for | Alloc Rec, S', 'input': ('ecoinvent 3.6 cutoff', '4cb6770a27217

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Title: Writing activities to SQLite3 database:
  Started: 12/02/2020 12:48:36
  Finished: 12/02/2020 12:48:46
  Total time elapsed: 00:00:10
  CPU %: 49.70
  Memory %: 0.24
Created database: Agribalyse 1.3 - ecoinvent 3.6 cutoff
CPU times: user 16min 34s, sys: 22.3 s, total: 16min 56s
Wall time: 18min 4s


In [ ]:
co = bw.Database('CH consumption 1.0')
act = co.search('average consumption')[0]
demand = {act: 1}
method = ('IPCC 2013', 'climate change', 'GTP 100a')
act

In [ ]:
%%time
lca = bw.LCA(demand, method)
lca.lci()
lca.lcia()
lca.score

In [ ]:
bw.databases

In [22]:
from bw2calc import MonteCarloLCA
import brightway2 as bw
bw.projects.set_current("GSA for oases")
bb = bw.Database("ecoinvent 3.6 cutoff")
cc = bw.Database("Agribalyse 1.3 - ecoinvent 3.6 cutoff")
method = ("IPCC 2013", "climate change", "GTP 100a")
demand = {cc.random(): 1}
lca = bw.LCA(demand, method)
lca.lci()
lca.lcia()
lca.score

0.0

In [23]:
a = MonteCarloLCA(demand, method)
[next(a) for _ in range(5)]

[0.0, 0.0, 0.0, 0.0, 0.0]